# Topic 13 - Recurrent Neural Networks

## [Video Lecture](https://www.youtube.com/watch?v=6niqTuYFZLQ)

## 1 The basic recurrent neural network
As we've seen, standard Markov Models don't work for capturing long term dependencies when making models of sequences: the number of parameters grows exponentially in the degree of the model (how far we look back in the sequence in order to make predictions about the next thing in the sequence).  This naturally leads us to think about other potential architectures.  Alternatively, we could think about how to use a neural network as a means for language modelling, particularly since we now know how to encode text information, either via one hot encodings or through word embeddings.  For example, we could apply a standard multilayer perceptron to the last $n$ words in a sentence in order to produce the next one.  However, the problem with this is that sentences aren't always the same length: we may have too few inputs (imagine if we tried to predict the third word in a sentence, but our MLP expected ten features), or our fixed input size might be too short (imagine if we tried to predict the 15th word in a sentence, say, a pronoun, but the noun to which it referred was the first word in the sentence).  Multilayer perceptrons are powerful, but ultimately suffer from the baked-in assumption that they *know where the predictive information is stored* (note that this was the same reason that we decided to use CNNs as well).  Recurrent neural networks essentially attempt to circumvent these problems by adopting the sequential structure of a Markov model along with the flexibility of a neural network.

The basic equation for determining the internal state of an RNN is
$$
\mathbf{h}_t = \tanh(W_{hh} \mathbf{h}_t + W_{xh} \mathbf{x}_t),
$$
which is to say that it's just the same math as the hidden layer of a multilayer perceptron, but applied *recurrently*: rather than just compute the activation of a linear transformation of the input features $\mathbf{x}_t$, the activation is applied to the sum of that linear transformation *and* a linear transformation of the *previous state's* activation.  Why is this useful?  It provides context.  Because $\mathbf{h}_t$ has been built up sequentially by looking at all of the previous states in the sequence, it has the capacity (for judicious parameter choices!) to distill all of that previous information into a representation that is actionable, particularly when combined with the current input.  

What is meant by actionable?  As usual $h_t$ is of arbitrary dimension: in order to make predictions, we simply complete the neural network by adding an additional linear transformation to create some outputs:
$$
\mathbf{y}_t = W_{hy} \mathbf{h}_t,
$$
which could of course be interpreted as logits (to be fed into softmax and cross-entropy) or as reals (to be fed into SSE).  As such, the graph of this RNN (a so-called Elman network, after it's original author) looks like this:
<img src="elman.svg" />
Just like a normal MLP, except for the linkages from the hidden nodes back to themselves.  For training, the RNN can be *unrolled*, such that we have the situation that
<img src="unrolled.svg" />
All the boxes represent one application of the hidden input augmented multilayer perceptron, with the additional note that all of the models are identical and share their weights.  Note that in this drawing, all the arrows go up and right: there are no loops, and thus this is a directed acyclic graph for which automatic differentiation works fine.  

For concreteness, if we were training a word prediction model, the input $x_t$ would be the encoding of the $t-$th word in the sequence, $y_t$ would be an unscaled log probability (logit) of the next word in the sentence (most likely to be compared to the real next word in a sentence), and $h_t$ represents some internal information distilled from all the inputs that have occurred before.

## 2 RNNs don't really work
Unfortunately, this specific architecture doesn't work that well in practice.  This is primarily due to the depth of the network, which yields a so-called *vanishing gradient problem*: as misfit information from later positions in the sequence propogates back through the network to interact with input information from earlier, the influence of any particular weight dies out and training fails.  Alternatively, repeated applications of matrix multiplication when weights are large can lead to something called the *exploding gradient problem*, where weights go to infinity via exponentially growing positive gradients.

## 3 Long short term memory
Long Short Term Memory networks(LSTMs) were designed to have the advantages of RNNs, but without the problematic behavior of exploding and vanishing gradients.  They have become the de facto standard mechanism for sequence modelling (until very recently, when something called a Transformer has supplanted them).  From a very high level view, the figure directly above, where a series of models with shared weights are applied to some input and some internal state in order to produce an output and an updated internal state that is passed to the next model in the sequence, holds.  The only major difference is in the specific operations performed inside that box, which are a little bit more complicated than the basic RNN

LSTM work by introducing a mechanism for explicitly remembering and forgetting pieces of information, which are stored in a second internal state called the cell state, which we will call $\mathbf{c}_t$.  Rather than reproduce the figures here, have a look at [this excellent blog post by Christopher Olah](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)    

## 4 Shakespeare simulator 2: Character-level boogaloo
While we spent a bit of time thinking about how to encode *words* as numbers in order to input them into a neural network, for understanding the expressive power of LSTMs, it's useful to step back just a bit and do something simpler: character level modelling.  Why should we do this, even though we have the capacity to embed words?  First, it's quite a bit simpler.  The vocabulary of characters is short, just the 26 letters of the alphabet, the various puncuation marks, and the whitespace.  Second, using characters, it's easier to see just how far the memory of this thing has to be in order to produce cogent results: if a word model has to look back several words to determine which pronoun or adjective is appropriate, a character model has to look back perhaps hundreds of characters.  We'll see that this is indeed possible.  

First, let's load a corpus of text, just as we did for Markov Models, but also we'll create tokenization dictionaries.

In [ ]:
from __future__ import division,print_function

import torch
from torch import nn
import numpy as np
#import string

import re


sequence_shakespeare = []
file = open('t8.shakespeare.txt','r')
for line in file:
    line.strip('\n')
    if line[:2] == '  ':
        line_words = re.findall(r"[\w']+|[.,!?;]",line)
        line_words = [str(w).lower() for w in line_words if not w.isupper() and not w.isdigit()] 

        sequence_shakespeare.extend(line_words)
        
print (' '.join(sequence_shakespeare[:100]))
text = ' '.join(sequence_shakespeare)
seq_size = 100
texts = []
for i in range(len(text)//100):
    texts.append(text[i*100:(i*100)+100])

# Limit ourselves to the first 100000 characters
text = texts[:1000]

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

Note that we've already batched our sequences: we've produced 1000 sequences each of 100 characters.

In [ ]:
text

Once again, with character level modelling, we have a relatively small vocabulary

In [ ]:
int2char

Now, we can split our sequences into inputs and targets: **if we are trying to predict the next letter in the sequence, what should our target sequence be?**

In [ ]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    input_seq.append(text[i][:-1])
    target_seq.append(text[i][1:])

for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

print("Input Sequence: {}\nTarget Sequence:  {}".format(input_seq[i], target_seq[i]))

Now that we have tokenized input, we can create one hot encodings of our inputs (note that we'll be using categorical cross entropy as a cost function, which in pytorch expects integer labels rather than one-hot encodings: thus we will not one-hot encode our labels.

In [ ]:
dict_size = len(char2int)
seq_len = len(input_seq[0])
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Create a one hot encoding with batch as the first axis, position in sequence along the second axis,
    # and one-hot vector along the third axis
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [ ]:
# Input shape --> (Batch Size, Sequence Length, One-Hot Encoding Size)
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)

In [ ]:
input_seq.shape

In [ ]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

If possible, we'd like to run this on a GPU.

In [ ]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

Now, we can define our model.  This proceeds much like it did for MLPs and CNNs: we subclass the Module object with an __init__ method and a forward method.

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        """ 
        Inputs: input_size (vocabulary size)
                output_size (also vocabulary size
                hidden_dim (size of the hidden state
                n_layers (depth of RNN layer)
        """
        super(Model, self).__init__()
  
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer - (GRU is strongly related to LSTM but works better for smaller datasets)
        self.rnn = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True,dropout=0.5)   
        
        # Fully connected layer - W_hy
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x, hidden):
        """
        Inputs: x (the sequence)
                hidden (the initial hidden layer state)
        Outputs: out (logits)
                 hidden (the final hidden layer state)
                """
        
        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs for input into fully connected layer: essentially this means that 
        # the batch and the sequence position get flattened into one axis, which is fine because we're 
        # going to take the sum anyways
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).cuda()
        return hidden

Now we can instatiate the model and send it to the GPU:

In [ ]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=256, n_layers=2)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.cuda()

Next, we'll set up our loss function and optimizer:

In [ ]:
# Define hyperparameters
n_epochs = 2000
lr=0.002

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Next, we'll wrap our dataset in a DataLoader to facilitate mini-batch gradient descent

In [ ]:
import torch.utils.data as data_utils

train = data_utils.TensorDataset(input_seq.cuda(), target_seq.cuda())
train_loader = data_utils.DataLoader(train, batch_size=1024, shuffle=False)

Because we'll want to see the quality of the language predictions as the model proceeds, we'll need to produce a method that creates random strings of characters (according to the model), based on a supplied initial string. 

In [ ]:
def predict(model, hidden, character, temperature):
    # Produce next character as function of current character and hidden state
    
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character).cuda()
    
    out, hidden = model(character,hidden)

    prob = nn.functional.softmax(out[-1]/temperature, dim=0).data
    char_ind = torch.multinomial(prob, 1).item()

    return int2char[char_ind], hidden

def sample(model, out_len, start='thy',temperature=1.0):
    # Create a string of out_len characters by repeated model prediction on the generated sequence.    
    model.eval() # eval mode
    start = start.lower()
    chars = [ch for ch in start]
    size = out_len - len(chars)
    hidden = model.init_hidden(1)
    for ii in range(size):
        char, hidden = predict(model, hidden, chars,temperature)
        chars.append(char)
    model.train()

    return ''.join(chars)

Finally, we can train our model.  This is, of course, highly familiar syntax, with the one exception being that we'll initialize hidden states with the final hidden state from the previous epoch.  

In [ ]:
# Training Run
model.train()

for epoch in range(1, n_epochs + 1):
    hidden = model.init_hidden(batch_size)
    for d,t in train_loader:
        optimizer.zero_grad()
        output, hidden = model(d,hidden)
        loss = criterion(output, t.view(-1).long())
        loss.backward() 
        optimizer.step()
    
    if epoch%10 == 0 or epoch==1:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))
        print(sample(model, 100, 'thy'))
        

In [ ]:
sample(model, 300, 'to',temperature=0.1)